# 03_06_debugging.ipynb - Debugging  with ipdb

In [ ]:
import math

In [ ]:
import numpy as np
import scipy
import scipy.integrate as si
import matplotlib.pyplot as pp

In [ ]:
import astropy
import astropy.time
import astropy.coordinates
import astropy.units
import astropy.constants

In [ ]:
# IAU 2012 values from from http://maia.usno.navy.mil/NSFA/NSFA_cbe.html

bodies = ['sun','mercury','venus','earth','mars','jupiter','saturn','uranus','neptune']

# dictionary of masses
massdict = {'sun': 1.0,
            'mercury': 1.6601209949637026e-07,
            'venus': 2.4478382857373332e-06,
            'earth': 3.0034896946063695e-06,
            'mars': 3.227156037857755e-07,
            'jupiter': 0.0009547918983127075,
            'saturn': 0.00028588567008942334,
            'uranus': 4.3662495719438076e-05,
            'neptune': 5.151383713179197e-05}

# array of masses
masses = np.array([massdict[body] for body in bodies])

In [ ]:
G = astropy.constants.G.to('AU^3 / (Msun d^2)')

In [ ]:
t0, t1 = astropy.time.Time('2021-07-04'), astropy.time.Time('2031-07-04')

In [ ]:
def get_posvel(body, t):
    posvel = astropy.coordinates.get_body_barycentric_posvel(body, t)
    
    return np.hstack([posvel[0].xyz.value.T, posvel[1].xyz.value.T])

In [ ]:
y0 = np.array([get_posvel(body, t0) for body in bodies])

In [ ]:
def ydot(t, y):
    # how many bodies? make sure the answer is an integer
    n = int(y.shape[0] / 6)

    # make an empty container for the derivatives
    yd = np.zeros_like(y)
    
    # for each body
    for i in range(n):
        # set x_i' = v_i (array slice assignment)
        yd[i*6:i*6+3] = y[i*6+3:i*6+6]
        
        # loop over all other bodies
        for j in range(n):
            if i == j:
                continue

            # add contribution of planet j to v_i'
            rij = y[j*6:j*6+3] - y[i*6:i*6+3]
            yd[i*6+3:i*6+6] += G * masses[j] * rij / np.dot(rij,rij)
    
    return yd

In [ ]:
ydot(t0, y0)

In [ ]:
ydot(t0, y0).shape

In [ ]:
%debug ydot(t0, y0)

In [ ]:
y0 = np.array([get_posvel(body, t0) for body in bodies]).flatten()

In [ ]:
y0.shape

In [ ]:
ydot(t0, y0)

In [ ]:
%debug

In [ ]:
G = astropy.constants.G.to('AU^3 / (Msun d^2)').value

In [ ]:
ydot(t0, y0)

In [ ]:
orbits = scipy.integrate.solve_ivp(ydot, [t0.mjd, t1.mjd], y0, rtol=1e-9, atol=1e-9)

In [ ]:
for i in range(9):
    pp.plot(orbits.y[i*6,:], orbits.y[i*6+1,:], label=bodies[i])

pp.legend()
pp.axis('equal');

In [ ]:
pp.plot(orbits.y[5*6,:], orbits.y[5*6+1,:])
pp.axis('equal');

In [ ]:
def ydot(t, y):
    # how many bodies? make sure the answer is an integer
    n = int(y.shape[0] / 6)

    # make an empty container for the derivatives
    yd = np.zeros_like(y)
    
    # for each body
    for i in range(n):
        # set x_i' = v_i (array slice assignment)
        yd[i*6:i*6+3] = y[i*6+3:i*6+6]
        
        # loop over all other bodies
        for j in range(n):
            if i == j:
                continue

            # add contribution of planet j to v_i'
            rij = y[j*6:j*6+3] - y[i*6:i*6+3]
            yd[i*6+3:i*6+6] += G * masses[j] * rij / np.dot(rij,rij)
    
    return yd

In [ ]:
%debug ydot(t0, y0)

In [ ]:
def ydot(t, y):
    # how many bodies? make sure the answer is an integer
    n = int(y.shape[0] / 6)

    # make an empty container for the derivatives
    yd = np.zeros_like(y)
    
    # for each body
    for i in range(n):
        # set x_i' = v_i (array slice assignment)
        yd[i*6:i*6+3] = y[i*6+3:i*6+6]
        
        # loop over all other bodies
        for j in range(n):
            if i == j:
                continue

            # add contribution of planet j to v_i'
            rij = y[j*6:j*6+3] - y[i*6:i*6+3]
            yd[i*6+3:i*6+6] += G * masses[j] * rij / np.dot(rij,rij)**1.5
    
    return yd

In [ ]:
orbits = scipy.integrate.solve_ivp(ydot, [t0.mjd, t1.mjd], y0, rtol=1e-9, atol=1e-9)

In [ ]:
for i in range(9):
    pp.plot(orbits.y[i*6,:], orbits.y[i*6+1,:], label=bodies[i])

pp.legend()
pp.axis('equal');